In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Nov 11 18:08:17 2018

@author: Arif Dinulsyah Putra
source : https://www.gdeltproject.org
"""

import requests
import lxml.html as lh
import os.path
import urllib.request
import zipfile
import glob
import operator
import pandas as pd
from subprocess import call
from sqlalchemy import create_engine
import pandas as pd
import pymysql

#function for obtaining files in GDELT Event 1.0
def get_files():
    months=['01','02','03','04','05','06','07','08','09','10']
    gdelt_base_url = 'http://data.gdeltproject.org/events/'
    page = requests.get(gdelt_base_url+'index.html')    
    doc = lh.fromstring(page.content)
    link_list = doc.xpath("//*/ul/li/a/@href")
    local_path='/Users/Project/Desktop/gdelt/'
    # separate out those links that begin in january until october in 2018
    file_list = [x for x in link_list if str(x[0:4]) == '2018' and str(x[4:6]) in months]
    
    infilecounter = 0
    
    for compressed_file in file_list[infilecounter:]:
    # if we dont have the compressed file stored locally, go get it. Keep trying if necessary.
        while not os.path.isfile(local_path + compressed_file):
            print('downloading, '),
            urllib.request.urlretrieve(url=gdelt_base_url + compressed_file,
                           filename=local_path + compressed_file)
            print('extracting,'),
            z = zipfile.ZipFile(file=local_path + compressed_file, mode='r')
            z.extractall(path=local_path + 'tmp/')
    
 #function for appending row/data with keyword --> trade war on URL attribute           
def parsing():
    keyword = 'trade-war' 
    local_path='/Users/Project/Desktop/gdelt/' # use your path
    outfilecounter=0
    # parse each of the csv files in the working directory,
    print('parsing....'),
    print('total file: ',len(glob.glob(local_path+'tmp/*')))
    for infile_name in glob.glob(local_path+'tmp/*'):
        print("processing.... "+ infile_name)
        #Make a directory/folder named gdeltdata and put also a plain/blank file named 0000.tsv on that directory
        outfile_name = local_path + 'gdeltdata/' + '%04i.tsv' % outfilecounter
        # open the infile and outfile
        with open(infile_name,mode='r') as infile, open(outfile_name, mode='w') as outfile:
            for line in infile:
            # extract lines with our interest
                if keyword in operator.itemgetter(57)(line.split('\t')):
                    outfile.write(line)
            outfilecounter += 1
            
#(FOR MAC OS)can try/find other command in windows for merged files
def merged_files():
    script="cat /Users/Project/Desktop/gdelt/gdeltdata/*.tsv > /Users/Project/Desktop/gdelt/gdeltdata/merged.tsv"
    call(script,shell=True)

def run(sql):
    df = pd.read_sql_query(sql,engine)
    return df
    
if __name__ == '__main__':
    #UNCOMMENT THIS SECTION TO RUN THE CODE
    #get_files() <---
    #parsing()  <---
    #merged_files() <---
    #UNCOMMENT THIS SECTION TO RUN THE CODE
    
    #This section for importing files to sql database 
    header = pd.read_csv('/Users/Project/Desktop/gdelt/gdeltdata/CSV.header.fieldids.csv')
    header_list=list()
    for c in header:
        header_list.append(c)
    engine=create_engine('mysql+pymysql://root@localhost:3306/gdelt')
    data_csv = pd.read_csv('/Users/Project/Desktop/gdelt/gdeltdata/merged.tsv',sep='\t',names=header_list)
    data_csv.to_sql(name='gdelt',con=engine,if_exists='replace',index=False)

# Query langsung di mysql database (Laragon/xampp) hasilnya 3000 an bisa di export ke csv kalau pake sql alchemy load hasilnya sedikit. URLnya banyak yg duplicate coba2 aja dulu

SELECT * FROM gdelt WHERE (Actor1Code='USA'and Actor2Code='CHN') OR (Actor1Code='CHN'and Actor2Code='USA') GROUP BY SOURCEURL

In [37]:
df = pd.read_csv('/Users/Project/Desktop/gdelt/gdeltdata/gdelt.csv',names=header_list)

In [41]:
df['AvgTone'].count()

3430

In [42]:
df['AvgTone'].min()

-17.5438596491228

In [43]:
df['AvgTone'].max()

4.25531914893617

In [45]:
df['AvgTone'].mean()

-2.998077666584076